In [ ]:
import requests
import pandas as pd
import json
import posixpath

def extract_URLs(json_data):
    """
    Extracts all URLs from the given JSON data.

    Args:
        json_data: A JSON string or a Python dictionary representing the JSON data.

    Returns:
        A list of all URLs found in the JSON data.
    """
    data = json.loads(json_data) if isinstance(json_data, str) else json_data
    URLs = []

    for entry in data.get('feed', {}).get('entry', []):
        for link in entry.get('links', []):
            URL = link.get('href')
            if URL and URL.startswith("https") and URL.endswith((".tif", ".json", ".aux.xml", ".jpeg")):
                URLs.append(URL)

    return URLs

def ECOSTRESS_CMR_search(concept_id, lat, lon, start_date, end_date):
    """
    Searches the CMR API for granules matching the given parameters.

    Args:
        concept_id: The concept ID of the collection to search.
        lat: The latitude of the search area.
        lon: The longitude of the search area.
        start_date: The start date of the search period.
        end_date: The end date of the search period.

    Returns:
        A pandas DataFrame with columns for orbit, scene, tile, and URL.
    """

    url = "https://cmr.earthdata.nasa.gov/search/granules.json"
    params = {
        "concept_id": concept_id,
        "bounding_box": f"{lon},{lat},{lon},{lat}",  # Point search
        "temporal": f"{start_date}T00:00:00Z,{end_date}T23:59:59Z",
        "page_size": 2000  # Adjust if needed
    }

    response = requests.get(url, params=params)
    response.raise_for_status()  # Raise an exception for bad status codes

    URLs = extract_URLs(response.json())

    records = []

    for URL in URLs:
        filename = posixpath.basename(URL)
        variable = ""

        if filename.endswith(".json"):
            granule_name = filename.split(".")[0]
        else:
            granule_name = "_".join(filename.split("_")[:-1])

        if filename.endswith((".tif", ".jpeg", ".jpeg.aux.xml")):
            variable = filename.split("_")[-1].split(".")[0]

        if filename.endswith(".json"):
            type = "JSON Metadata"
        elif filename.endswith(".tif"):
            type = "GeoTIFF Data"
        elif filename.endswith(".jpeg"):
            type = "GeoJPEG Preview"
        elif filename.endswith(".jpeg.aux.xml"):
            type = "GeoJPEG Metadata"
        else:
            raise ValueError(f"Unknown file type for {filename}")

        try:
            product = "_".join(granule_name.split("_")[1:-6])
            orbit = int(granule_name.split("_")[-6])
            scene = int(granule_name.split("_")[-5])
            tile = granule_name.split("_")[-4]
            records.append({
                "product": product,
                "variable": variable,
                "orbit": orbit, 
                "scene": scene, 
                "tile": tile, 
                "type": type,
                "granule": granule_name,
                "filename": filename,
                "URL": URL
            })
        except (IndexError, ValueError):
            print(f"Filename {filename} does not match expected pattern and was skipped.")

    df = pd.DataFrame(records, columns=[
        "product", 
        "variable", 
        "orbit", 
        "scene",
        "tile", 
        "type", 
        "granule", 
        "filename", 
        "URL"
    ])

    return df

# Parameters for search
concept_id = "C2076106409-LPCLOUD"
lat = 34
lon = -118
start_date = "2024-10-10"
end_date = "2024-11-11"

# Perform search and display DataFrame
df = ECOSTRESS_CMR_search(concept_id, lat, lon, start_date, end_date)
df

,product,variable,orbit,scene,tile,granule,filename,URL
0,L3T_JET,STICinst,35510,5,11SMT,ECOv002_L3T_JET_35510_005_11SMT_20241010T15165...,ECOv002_L3T_JET_35510_005_11SMT_20241010T15165...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...
1,L3T_JET,cloud,35510,5,11SMT,ECOv002_L3T_JET_35510_005_11SMT_20241010T15165...,ECOv002_L3T_JET_35510_005_11SMT_20241010T15165...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...
2,L3T_JET,PTJPLSMinst,35510,5,11SMT,ECOv002_L3T_JET_35510_005_11SMT_20241010T15165...,ECOv002_L3T_JET_35510_005_11SMT_20241010T15165...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...
3,L3T_JET,BESSinst,35510,5,11SMT,ECOv002_L3T_JET_35510_005_11SMT_20241010T15165...,ECOv002_L3T_JET_35510_005_11SMT_20241010T15165...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...
4,L3T_JET,MOD16inst,35510,5,11SMT,ECOv002_L3T_JET_35510_005_11SMT_20241010T15165...,ECOv002_L3T_JET_35510_005_11SMT_20241010T15165...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...
...,...,...,...,...,...,...,...,...
513,L3T_JET,water,35820,12,11SMT,ECOv002_L3T_JET_35820_012_11SMT_20241030T15184...,ECOv002_L3T_JET_35820_012_11SMT_20241030T15184...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...
514,L3T_JET,PTJPLSMcanopy,35820,12,11SMT,ECOv002_L3T_JET_35820_012_11SMT_20241030T15184...,ECOv002_L3T_JET_35820_012_11SMT_20241030T15184...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...
515,L3T_JET,STICcanopy,35820,12,11SMT,ECOv002_L3T_JET_35820_012_11SMT_20241030T15184...,ECOv002_L3T_JET_35820_012_11SMT_20241030T15184...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...
516,L3T_JET,PTJPLSMsoil,35820,12,11SMT,ECOv002_L3T_JET_35820_012_11SMT_20241030T15184...,ECOv002_L3T_JET_35820_012_11SMT_20241030T15184...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...


In [33]:
list(df[df.URL.apply(lambda URL: URL.endswith(".jpeg.aux.xml"))].filename)

['ECOv002_L3T_JET_35510_005_11SMT_20241010T151659_0713_01_cloud.jpeg.aux.xml',
 'ECOv002_L3T_JET_35510_005_11SMT_20241010T151659_0713_01_STICinst.jpeg.aux.xml',
 'ECOv002_L3T_JET_35510_005_11SMT_20241010T151659_0713_01_PTJPLSMinst.jpeg.aux.xml',
 'ECOv002_L3T_JET_35510_005_11SMT_20241010T151659_0713_01_BESSinst.jpeg.aux.xml',
 'ECOv002_L3T_JET_35510_005_11SMT_20241010T151659_0713_01_MOD16inst.jpeg.aux.xml',
 'ECOv002_L3T_JET_35510_005_11SMT_20241010T151659_0713_01_ETdaily.jpeg.aux.xml',
 'ECOv002_L3T_JET_35510_005_11SMT_20241010T151659_0713_01_ETinstUncertainty.jpeg.aux.xml',
 'ECOv002_L3T_JET_35510_005_11SMT_20241010T151659_0713_01_PTJPLSMcanopy.jpeg.aux.xml',
 'ECOv002_L3T_JET_35510_005_11SMT_20241010T151659_0713_01_STICcanopy.jpeg.aux.xml',
 'ECOv002_L3T_JET_35510_005_11SMT_20241010T151659_0713_01_PTJPLSMsoil.jpeg.aux.xml',
 'ECOv002_L3T_JET_35510_005_11SMT_20241010T151659_0713_01_water.jpeg.aux.xml',
 'ECOv002_L3T_JET_35510_005_11SMT_20241010T151659_0713_01_PTJPLSMinterception.jpeg

In [34]:
granule_df = df[(df.orbit == 35510) & (df.scene == 5) & (df.tile == "11SMT")]
granule_df

,product,variable,orbit,scene,tile,granule,filename,URL
0,L3T_JET,STICinst,35510,5,11SMT,ECOv002_L3T_JET_35510_005_11SMT_20241010T15165...,ECOv002_L3T_JET_35510_005_11SMT_20241010T15165...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...
1,L3T_JET,cloud,35510,5,11SMT,ECOv002_L3T_JET_35510_005_11SMT_20241010T15165...,ECOv002_L3T_JET_35510_005_11SMT_20241010T15165...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...
2,L3T_JET,PTJPLSMinst,35510,5,11SMT,ECOv002_L3T_JET_35510_005_11SMT_20241010T15165...,ECOv002_L3T_JET_35510_005_11SMT_20241010T15165...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...
3,L3T_JET,BESSinst,35510,5,11SMT,ECOv002_L3T_JET_35510_005_11SMT_20241010T15165...,ECOv002_L3T_JET_35510_005_11SMT_20241010T15165...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...
4,L3T_JET,MOD16inst,35510,5,11SMT,ECOv002_L3T_JET_35510_005_11SMT_20241010T15165...,ECOv002_L3T_JET_35510_005_11SMT_20241010T15165...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...
5,L3T_JET,ETdaily,35510,5,11SMT,ECOv002_L3T_JET_35510_005_11SMT_20241010T15165...,ECOv002_L3T_JET_35510_005_11SMT_20241010T15165...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...
6,L3T_JET,water,35510,5,11SMT,ECOv002_L3T_JET_35510_005_11SMT_20241010T15165...,ECOv002_L3T_JET_35510_005_11SMT_20241010T15165...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...
7,L3T_JET,ETinstUncertainty,35510,5,11SMT,ECOv002_L3T_JET_35510_005_11SMT_20241010T15165...,ECOv002_L3T_JET_35510_005_11SMT_20241010T15165...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...
8,L3T_JET,PTJPLSMcanopy,35510,5,11SMT,ECOv002_L3T_JET_35510_005_11SMT_20241010T15165...,ECOv002_L3T_JET_35510_005_11SMT_20241010T15165...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...
9,L3T_JET,STICcanopy,35510,5,11SMT,ECOv002_L3T_JET_35510_005_11SMT_20241010T15165...,ECOv002_L3T_JET_35510_005_11SMT_20241010T15165...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...


In [35]:
list(granule_df.filename)

['ECOv002_L3T_JET_35510_005_11SMT_20241010T151659_0713_01_STICinst.tif',
 'ECOv002_L3T_JET_35510_005_11SMT_20241010T151659_0713_01_cloud.tif',
 'ECOv002_L3T_JET_35510_005_11SMT_20241010T151659_0713_01_PTJPLSMinst.tif',
 'ECOv002_L3T_JET_35510_005_11SMT_20241010T151659_0713_01_BESSinst.tif',
 'ECOv002_L3T_JET_35510_005_11SMT_20241010T151659_0713_01_MOD16inst.tif',
 'ECOv002_L3T_JET_35510_005_11SMT_20241010T151659_0713_01_ETdaily.tif',
 'ECOv002_L3T_JET_35510_005_11SMT_20241010T151659_0713_01_water.tif',
 'ECOv002_L3T_JET_35510_005_11SMT_20241010T151659_0713_01_ETinstUncertainty.tif',
 'ECOv002_L3T_JET_35510_005_11SMT_20241010T151659_0713_01_PTJPLSMcanopy.tif',
 'ECOv002_L3T_JET_35510_005_11SMT_20241010T151659_0713_01_STICcanopy.tif',
 'ECOv002_L3T_JET_35510_005_11SMT_20241010T151659_0713_01_PTJPLSMsoil.tif',
 'ECOv002_L3T_JET_35510_005_11SMT_20241010T151659_0713_01_PTJPLSMinterception.tif',
 'ECOv002_L3T_JET_35510_005_11SMT_20241010T151659_0713_01_cloud.jpeg.aux.xml',
 'ECOv002_L3T_JET_